In [1]:
import sys

# This package is not configured properly, I think.
# Without this line, all methods & submodules will fail to import
sys.path.append('./abstract_algebra/src')

from typing import Union
from functools import reduce
from random import randint
import pathlib
import polars as pl
import os


from abstract_algebra.src.finite_algebras import generate_symmetric_group

In [3]:
data_dir = pathlib.Path('./data')
if not os.path.exists(data_dir):
  os.mkdir(data_dir)

In [3]:
S5 = generate_symmetric_group(
  5, 
  name="S5", 
  description="Symmetric group on 5 elements")
S5.about(use_table_names=True)


** Group **
Name: S5
Instance ID: 4397950832
Description: Symmetric group on 5 elements
Order: 120
Identity: (1, 2, 3, 4, 5)
Commutative? No
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0 (1, 2, 3, 4, 5) (1, 2, 3, 4, 5)       1
      1 (1, 2, 3, 5, 4) (1, 2, 3, 5, 4)       2
      2 (1, 2, 4, 3, 5) (1, 2, 4, 3, 5)       2
      3 (1, 2, 4, 5, 3) (1, 2, 5, 3, 4)       3
      4 (1, 2, 5, 3, 4) (1, 2, 4, 5, 3)       3
      5 (1, 2, 5, 4, 3) (1, 2, 5, 4, 3)       2
      6 (1, 3, 2, 4, 5) (1, 3, 2, 4, 5)       2
      7 (1, 3, 2, 5, 4) (1, 3, 2, 5, 4)       2
      8 (1, 3, 4, 2, 5) (1, 4, 2, 3, 5)       3
      9 (1, 3, 4, 5, 2) (1, 5, 2, 3, 4)       4
     10 (1, 3, 5, 2, 4) (1, 4, 2, 5, 3)       4
     11 (1, 3, 5, 4, 2) (1, 5, 2, 4, 3)       3
     12 (1, 4, 2, 3, 5) (1, 3, 4, 2, 5)       3
     13 (1, 4, 2, 5, 3) (1, 3, 5, 2, 4)       4
     14 (1, 4, 3, 2, 5) (1, 4, 3, 2, 5)       2
     15 (1, 4, 3, 5, 2) (1, 5, 3, 2, 4)       3
     16 (1, 4, 5, 2, 3) (1, 4, 5, 2

In [17]:
def group_reduce(lhs: Union[str, int], rhs: int) -> int:
  if isinstance(lhs, str):
    prod = S5.op(lhs, S5.elements[rhs])
  else:
    prod = S5.op(S5.elements[lhs], S5.elements[rhs])
  
  return S5.elements.index(prod)


examples = []
num_examples = 10_000
max_len = 20
for prod_len in range(1, max_len+1):
  for i in range(num_examples):
    lhs = []
    for _ in range(prod_len):
      lhs.append(randint(0, len(S5.elements)-1))
    examples.append({"length": len(lhs), "input": " ".join(map(str, lhs)), "target": str(reduce(group_reduce, lhs))})


In [23]:
ex_df = pl.from_dicts(examples)

for i in range(1, max_len+1):
  ex_df.filter(pl.col("length") == i).write_csv(data_dir / f"S5_{i}.csv")

In [19]:
ex_df.tail()

length,input,target
i64,str,str
20,"""113 58 70 29 3…","""111"""
20,"""84 24 31 46 48…","""105"""
20,"""22 97 40 36 67…","""56"""
20,"""106 3 19 13 10…","""105"""
20,"""27 98 51 108 9…","""50"""
